In [7]:
import os
import sys
import shutil
import pandas
import numpy
import ipystata

In [8]:
from ipystata.config import config_stata
config_stata('/home/kmongeon/research/stata14/stata-mp',force_batch=True)

In [9]:
%%stata --openstata
display "Test Stata"


Test Stata


In [10]:
# %%stata 
# ssc install tabout 
# ssc install estout

In [11]:
os.getcwd()

'/home/kmongeon/research/exercise_physiology/scripts'

In [12]:
os.listdir()

['sem_march2017.do',
 'sem_april2017.do',
 '.ipynb_checkpoints',
 'gsem_may_2017.ipynb',
 'data',
 'clean_data_march2017.do',
 'sem_march2017_bayes.do',
 'out']

In [13]:
shutil.rmtree('out')

In [14]:
os.mkdir('out')

In [15]:
dr = pandas.read_csv('./data/IzzyLongitudinalWorkingMarch2015.csv')
dt = pandas.read_csv('./data/torque.csv')
dn = pandas.read_csv('./data/NegativeIDs.csv')
df = pandas.read_csv('./data/data_examine_falk_nov2016.csv')

In [17]:
dr.columns

Index(['ID', 'Session', 'Sequence', 'Gender', 'Age', 'Height', 'Seated Height',
       'Leg Length', 'Weight', 'BF_Slaughter', 'BIA_PBF', 'BMI', 'LBM (kg)',
       'Years from aPHV', 'Age_At_PHV', 'Mat_Offset_New', 'Bone_Age', 'Tanner',
       'Menarche', 'Age_First_Period', 'Period_Frequency', 'Medications',
       'Vitamins', 'Fracture', 'Radius_SOS', 'Radius_Z', 'Tibial_SOS',
       'Tibial_Z', 'Wrist_Flexion_Best', 'Grip_Strength_Best',
       'Forearm_Length', 'ForeArm_Lean_CSA', 'Forearm_US_CSA', 'Anterior_CSA',
       'Anterior Radius', 'Ant_Rad_CSA', 'Anterior Ulna', 'Ant_Ulna_CSA',
       'Posterior', 'Post_CSA', 'Tibialis Anterior', 'NTx', 'Creatinine',
       'NTx/Creat', 'Caloric Intake', 'Calcium', 'Vit D', 'Vit D.1',
       'Caffeine', 'tot sedetary', 'tot light', 'tot mod', 'tot hard',
       'tot v hard', 'tot mvh', 'Total time', 'Godin_PA', 'PAQ_TOTAL_SCORE',
       'PAQ-PRVNT', 'Training_Data', 'Upper Arm_Length', 'Arm_Circumference',
       'Arm_Lean_CSA', 'Thigh_Len

In [18]:
dt.columns

Index(['ID', 'Biodex_ID', 'Sequence', 'Session', 'New_Sequence', 'Gender',
       'Visit_One', 'Visit_Two', 'DOB', 'Age (days)', 'Age (years)', 'ptiso',
       'pt60', 'pt240'],
      dtype='object')

In [19]:
dn.columns
dn = dn[['nota', 'ID', 'Sequence']].rename(columns={'ID': 'id', 'Session': 'ses', 'Sequence': 'sequence'})


In [20]:
df.columns
df = df[['falk', 'id', 'sequence']]

In [21]:
dr = dr[['ID', 'Session', 'Sequence', 'Radius_SOS', 'Tibial_SOS', 'Gender', 'Mat_Offset_New', 'Grip_Strength_Best', 'NTx/Creat', 'tot mvh']]
dr = dr.rename(columns={'ID': 'id', 'Session': 'ses', 'Sequence': 'sequence', 'Radius_SOS': 'rsos', 'Tibial_SOS': 'tsos', 'Gender': 'gender', 'Mat_Offset_New': 'matu', 'Grip_Strength_Best': 'grip', 'NTx/Creat': 'ntx', 'tot mvh': 'mvh'})
dr.columns

Index(['id', 'ses', 'sequence', 'rsos', 'tsos', 'gender', 'matu', 'grip',
       'ntx', 'mvh'],
      dtype='object')

In [22]:
dt = dt[['ID', 'Session', 'New_Sequence', 'ptiso']]
dt = dt.rename(columns={'ID': 'id', 'Session': 'ses', 'New_Sequence': 'sequence'})
dt.columns

Index(['id', 'ses', 'sequence', 'ptiso'], dtype='object')

In [23]:
dm = pandas.merge(dr, dt, on=['id', 'sequence'], how='outer', indicator=False)
dm = pandas.merge(dm, dn, on=['id', 'sequence'], how='left', indicator=False)
dm = pandas.merge(dm, df, on=['id', 'sequence'], how='left', indicator=False)

dm = dm.drop(dm[(dm.id==517) & (dm.sequence ==7)].index)
dm = dm.drop(dm[(dm.matu>5)].index)
dm = dm.drop(dm[(dm.matu.isnull())].index)
# dm.ptiso = numpy.where(dm.ptiso==-9999, ' ', dm.ptiso)
# dm.rsos = numpy.where(dm.nota==1, ' ', dm.rsos)
# dm.tsos = numpy.where(dm.falk==1, ' ', dm.tsos)

In [24]:
%%stata --data dm --output dm 

sort id sequence
by id: egen seq = seq() 
drop sequence
replace ptiso = . if ptiso==-9999
replace rsos = . if nota==1
replace tsos = . if falk==1 
order id seq ses_x ses_y rsos tsos grip ptiso ntx mvh matu gender nota falk


(64 real changes made, 64 to missing)
(50 real changes made, 50 to missing)
(11 real changes made, 11 to missing)


In [58]:
%%stata --data dm --output dm 

gsem ( grip <- rsos matu M1[id]) 
predict ghat

gsem ( ptiso <- tsos matu M1[id]) 
predict phat

gsem ( mvh <- tsos phat matu M1[id]) 
predict mhat

gsem ( ntx <- tsos phat matu mvh M1[id]) 
predict nhat


Fitting fixed-effects model:

Iteration 0:   log likelihood = -1003.6248  
Iteration 1:   log likelihood = -1003.6248  

Refining starting values:

Grid node 0:   log likelihood = -990.24397

Fitting full model:

Iteration 0:   log likelihood = -990.24397  (not concave)
Iteration 1:   log likelihood = -942.00898  (not concave)
Iteration 2:   log likelihood = -923.89042  
Iteration 3:   log likelihood = -908.94077  
Iteration 4:   log likelihood = -905.03584  
Iteration 5:   log likelihood = -904.78692  
Iteration 6:   log likelihood = -904.78684  

Generalized structural equation model           Number of obs     =        319
Response       : grip
Family         : Gaussian
Link           : identity
Log likelihood = -904.78684

 ( 1)  [grip]M1[id] = 1
------------------------------------------------------------------------------
             |      Coef.   Std. Err.      z    P>|z|     [95% Conf. Interval]
-------------+----------------------------------------------------------------
g

In [64]:
dm.describe()

,id,seq,ses_x,ses_y,rsos,tsos,grip,ptiso,ntx,mvh,matu,gender,nota,falk,ghat,phat,mhat,nhat
count,425.000000,425.000000,425.000000,374.000000,365.000000,404.000000,379.000000,422.000000,385.000000,307.000000,425.000000,425.000000,50.0,425.000000,365.000000,404.000000,404.000000,294.000000
mean,326.197647,1.868235,1.430588,1.486631,3819.567123,3686.920792,23.438074,124.876382,540.346624,106.180456,-0.893576,0.468235,1.0,0.025882,23.063364,125.209999,102.478943,565.195862
std,197.549787,0.877686,0.495742,0.500491,99.265290,107.642265,7.364498,53.080033,265.354984,41.705219,1.942006,0.499578,0.0,0.158971,7.176276,50.196468,31.668610,108.634354
min,100.000000,1.000000,1.000000,1.000000,3542.000000,3381.000000,9.000000,18.608421,56.308327,13.570000,-5.300000,0.000000,1.0,0.000000,7.403130,18.608421,22.031950,227.567596
25%,138.000000,1.000000,1.000000,1.000000,3754.000000,3618.750000,18.000000,84.669421,369.744916,78.140000,-2.290000,0.000000,1.0,0.000000,18.237528,89.378687,80.377050,501.744759
50%,189.000000,2.000000,1.000000,1.000000,3821.000000,3688.000000,22.250000,116.883744,514.754758,99.110000,-1.000000,0.000000,1.0,0.000000,21.828442,119.719856,101.931835,566.652283
75%,530.000000,2.000000,2.000000,2.000000,3878.000000,3757.250000,27.500000,156.184950,662.435602,133.390000,0.450000,1.000000,1.0,0.000000,27.506365,152.513237,124.966249,630.494415
max,583.000000,4.000000,2.000000,2.000000,4140.000000,4015.000000,53.000000,343.836793,1754.324517,235.000000,4.950000,1.000000,1.0,1.000000,49.581211,334.305695,195.993469,934.592407


In [63]:
%%stata --data dm --output dm

replace ptiso = phat if missing(ptiso)


(112 real changes made)


In [65]:
%%stata --data dm --output dm

gsem (tsos <- ptiso matu M1[id]) (ptiso <- matu  M2[id]) 




Fitting fixed-effects model:

Iteration 0:   log likelihood = -4532.4585  
Iteration 1:   log likelihood = -4532.4585  

Refining starting values:

Grid node 0:   log likelihood =  -4532.224

Fitting full model:

Iteration 0:   log likelihood =  -4532.224  (not concave)
Iteration 1:   log likelihood = -4364.6872  
Iteration 2:   log likelihood = -4337.7802  
Iteration 3:   log likelihood = -4324.5979  
Iteration 4:   log likelihood = -4309.0293  
Iteration 5:   log likelihood = -4308.4174  
Iteration 6:   log likelihood = -4308.4015  
Iteration 7:   log likelihood = -4308.4015  

Generalized structural equation model           Number of obs     =        422

Response       : tsos                           Number of obs     =        404
Family         : Gaussian
Link           : identity

Response       : ptiso                          Number of obs     =        422
Family         : Gaussian
Link           : identity

Log likelihood = -4308.4015

 ( 1)  [tsos]M1[id] = 1
 ( 2)  [ptiso]M

In [424]:
%%stata --data dm --output dm

gsem (rsos <- grip2  matu M1[id]) (grip2 <- matu )  ///
     (tsos <- ptiso2 matu M1[id]) (ptiso2 <- matu) 
predict mu*

gen rhat = mu1
gen ghat = mu2
gen that = mu3
gen phat = mu4

drop mu*

replace rsos  = rhat if missing(rsos)
replace grip  = ghat if missing(ghat)
replace tsos  = that if missing(that)
replace ptiso = phat if missing(phat)



>      (tsos <- ptiso2 matu M1[id]) (ptiso2 <- matu) 

Fitting fixed-effects model:

Iteration 0:   log likelihood =   -7903.87  
Iteration 1:   log likelihood =   -7903.87  

Refining starting values:

Grid node 0:   log likelihood = -7903.8484

Fitting full model:

Iteration 0:   log likelihood = -7903.8484  (not concave)
Iteration 1:   log likelihood = -7864.1113  (not concave)
Iteration 2:   log likelihood = -7863.2015  (not concave)
Iteration 3:   log likelihood = -7815.4882  (not concave)
Iteration 4:   log likelihood = -7798.5202  (not concave)
Iteration 5:   log likelihood = -7796.4903  (not concave)
Iteration 6:   log likelihood =  -7794.064  (not concave)
Iteration 7:   log likelihood = -7792.0706  (not concave)
Iteration 8:   log likelihood =  -7789.867  
Iteration 9:   log likelihood = -7786.0941  
Iteration 10:  log likelihood = -7777.0629  
Iteration 11:  log likelihood = -7772.2749  
Iteration 12:  log likelihood = -7771.3897  (not concave)
Iteration 13:  log likelihood

In [425]:
dm.head()

,id,seq,ses_x,ses_y,rsos,tsos,grip,ptiso,ntx,mvh,matu,gender,nota,falk,grip2,ptiso2,rhat,ghat,that,phat
0,100,1,1.0,1.0,3828.000000,3601.0,NaN,113.928056,711.848160,105.71,-1.67,0.0,NaN,0,27.406057,124.265144,3785.533447,53.265518,3629.462646,121.428062
1,100,2,1.0,1.0,3898.000000,3629.0,27.0,136.023758,760.093945,93.62,-0.71,0.0,NaN,0,30.634958,146.255936,3813.449707,55.639996,3663.475098,139.615829
2,100,3,1.0,1.0,3846.018799,3677.0,37.0,177.053081,543.734635,98.14,0.41,0.0,1.0,0,34.402008,171.911865,3846.018799,58.410217,3703.156494,160.834885
3,100,4,1.0,NaN,3952.000000,3740.0,40.5,NaN,454.364338,88.14,1.17,0.0,NaN,0,36.958221,189.321243,3868.119141,60.290009,3730.083008,175.233536
4,101,1,1.0,1.0,3682.000000,3603.0,NaN,133.048794,937.996794,89.29,-1.63,0.0,NaN,0,22.720345,127.405861,3762.957764,79.520111,3605.079590,130.438934


In [431]:
dm.describe()

,id,seq,ses_x,ses_y,rsos,tsos,grip,ptiso,ntx,mvh,...,grip2,ptiso2,rhat,ghat,that,phat,mu1,mu2,mu3,mu4
count,425.000000,425.000000,425.000000,374.000000,425.000000,404.000000,379.000000,310.000000,385.000000,307.000000,...,425.000000,425.000000,425.000000,425.000000,425.000000,425.000000,379.000000,425.000000,310.000000,425.000000
mean,326.197647,1.868235,1.430588,1.486631,3817.473514,3686.920792,23.438074,124.041289,540.346624,106.180456,...,23.043966,125.995918,3817.473633,20.293442,3687.312744,125.128036,3822.862061,13.259696,3679.259766,125.192513
std,197.549787,0.877686,0.495742,0.500491,94.280677,107.642265,7.364498,53.311306,265.354984,41.705219,...,6.870911,48.672268,63.735935,72.806572,90.222931,46.292458,76.088936,102.874443,75.893372,51.274364
min,100.000000,1.000000,1.000000,1.000000,3542.000000,3381.000000,9.000000,32.647165,56.308327,13.570000,...,8.087596,20.640436,3649.518066,-144.387741,3427.459961,3.859143,3602.933838,-235.380539,3493.343994,6.179320
25%,138.000000,1.000000,1.000000,1.000000,3758.370117,3618.750000,18.000000,84.335384,369.744916,78.140000,...,18.238676,91.644478,3775.400879,-25.686195,3629.462646,93.244278,3770.833984,-53.397549,3624.996277,88.540894
50%,189.000000,2.000000,1.000000,1.000000,3820.000000,3688.000000,22.250000,114.025640,514.754758,99.110000,...,22.252785,121.045067,3818.796143,17.608583,3684.965820,126.080498,3820.883545,4.335032,3681.825928,118.225365
75%,530.000000,2.000000,2.000000,2.000000,3874.000000,3757.250000,27.500000,156.607565,662.435602,133.390000,...,27.041368,154.021973,3859.623535,69.906425,3747.602051,158.481995,3875.070068,78.640533,3728.133850,155.280334
max,583.000000,4.000000,2.000000,2.000000,4140.000000,4015.000000,53.000000,343.836793,1754.324517,235.000000,...,48.182629,325.564667,4006.025879,192.825195,3957.858643,242.273529,4071.156250,293.898132,3944.708740,330.258209


In [433]:
%%stata --data dm 

gsem (rhat <- ghat matu M1[id]) (ghat  <- matu M2[id])


'Failed to open Stata'

In [429]:
%%stata --data dm --output dm

gsem (rsos <- grip  matu M1[id]) (grip  <- matu M2[id])  ///
     (tsos <- ptiso matu M1[id]) (ptiso <- matu M3[id])  
predict mu*


>      (tsos <- ptiso matu M1[id]) (ptiso <- matu M3[id])  

Fitting fixed-effects model:

Iteration 0:   log likelihood = -6705.2286  
Iteration 1:   log likelihood = -6705.2286  

Refining starting values:

Grid node 0:   log likelihood =  -6699.222

Fitting full model:

Iteration 0:   log likelihood =  -6699.222  (not concave)
Iteration 1:   log likelihood = -6674.6938  (not concave)
Iteration 2:   log likelihood = -6638.9712  (not concave)
Iteration 3:   log likelihood = -6631.8929  (not concave)
Iteration 4:   log likelihood = -6629.0683  (not concave)
Iteration 5:   log likelihood = -6626.8118  (not concave)
Iteration 6:   log likelihood = -6626.3607  (not concave)
Iteration 7:   log likelihood = -6626.3156  (not concave)
Iteration 8:   log likelihood = -6626.2976  (not concave)
Iteration 9:   log likelihood = -6626.2904  (not concave)
Iteration 10:  log likelihood =   -6626.29  (not concave)
Iteration 11:  log likelihood = -6626.2899  (not concave)
Iteration 12:  log likelihood

In [427]:
dm.to_csv('./out/dm.csv')